# Pyspark: Natural Language Processing

In [1]:
import findspark
findspark.init('/home/srabin/spark-3.5.1-bin-hadoop3')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

24/08/08 13:37:11 WARN Utils: Your hostname, DESKTOP-P0KTK6U resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/08/08 13:37:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/08 13:37:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/08 13:37:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/08/08 13:37:13 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/08/08 13:37:13 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


# Tokenization: 
## is the process of taking a text (sentence) and breaking it into individual terms, usually words

In [10]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer #regular expression tokenizer

In [3]:
from pyspark.sql.functions import col, udf

In [4]:
from pyspark.sql.types import IntegerType

In [16]:
sen_df = spark.createDataFrame([
    (0, 'Hi I heard about Spark'), 
    (1, 'I wish java could use case classes'),
    (2, 'Logistic,regression,models,are,neat')],
    ['id', 'sentence'])    

In [17]:
sen_df.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish java could...|
|  2|Logistic,regressi...|
+---+--------------------+



In [18]:
tokenizer = Tokenizer (inputCol = 'sentence', outputCol = 'words')

In [19]:
regex_tokenizer = RegexTokenizer(inputCol = 'sentence', outputCol = 'words', pattern = '\\W')

In [20]:
count_tokens = udf(lambda words: len(words), IntegerType())

In [21]:
tokenized = tokenizer.transform(sen_df)

In [22]:
tokenized.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|
|  1|I wish java could...|[i, wish, java, c...|
|  2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+



In [23]:
tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic,regress...|     1|
+---+--------------------+--------------------+------+



In [24]:
rg_tokenized = regex_tokenizer.transform(sen_df)

In [26]:
rg_tokenized.withColumn('tokens', count_tokens(col('words'))).show() 
# here it considers white spaces between words logestic, regression, ...

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic, regres...|     5|
+---+--------------------+--------------------+------+



# Stop words removal

In [27]:
from pyspark.ml.feature import StopWordsRemover

In [28]:
sentenceDataFrame = spark.createDataFrame([
    (0, ['I', 'saw', 'the', 'green', 'horse']),
    (1, ['Mary', 'had', 'a', 'little', 'lamb'])]
      ,['id', 'tokens'])

In [29]:
sentenceDataFrame.show()

+---+--------------------+
| id|              tokens|
+---+--------------------+
|  0|[I, saw, the, gre...|
|  1|[Mary, had, a, li...|
+---+--------------------+



In [30]:
remover = StopWordsRemover(inputCol = 'tokens', outputCol = 'filtered')

In [31]:
remover.transform(sentenceDataFrame).show()

+---+--------------------+--------------------+
| id|              tokens|            filtered|
+---+--------------------+--------------------+
|  0|[I, saw, the, gre...| [saw, green, horse]|
|  1|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+



## n-gram: sequence of n tokens takes the output of the tokennizer as input and then the parameter n is used to determine the number of terms in each n-gram. 

In [32]:
from pyspark.ml.feature import NGram

In [33]:
wordDataFrame= spark.createDataFrame([
    (0, ['Hi', 'I', 'heard', 'about', 'Spark']), 
    (1, ['I', 'wish', 'java', 'could', 'use', 'case', 'classes']),
    (2, ['Logistic', 'regression', 'models', 'are', 'neat'])],
    ['id', 'words'])    

In [34]:
ngram = NGram(n = 2, inputCol = 'words', outputCol = 'grams')

In [35]:
ngram.transform(wordDataFrame).show()

+---+--------------------+--------------------+
| id|               words|               grams|
+---+--------------------+--------------------+
|  0|[Hi, I, heard, ab...|[Hi I, I heard, h...|
|  1|[I, wish, java, c...|[I wish, wish jav...|
|  2|[Logistic, regres...|[Logistic regress...|
+---+--------------------+--------------------+



In [36]:
ngram.transform(wordDataFrame).select('grams').show(truncate = False)

+------------------------------------------------------------------+
|grams                                                             |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about Spark]                         |
|[I wish, wish java, java could, could use, use case, case classes]|
|[Logistic regression, regression models, models are, are neat]    |
+------------------------------------------------------------------+



In [37]:
# each of the above grams contains 2 words like Hi I, I heard since n = 2

In [39]:
ngram = NGram(n = 3, inputCol = 'words', outputCol = 'grams')
ngram.transform(wordDataFrame).select('grams').show(truncate = False)

+--------------------------------------------------------------------------------+
|grams                                                                           |
+--------------------------------------------------------------------------------+
|[Hi I heard, I heard about, heard about Spark]                                  |
|[I wish java, wish java could, java could use, could use case, use case classes]|
|[Logistic regression models, regression models are, models are neat]            |
+--------------------------------------------------------------------------------+

